# Developing Merge Utilities

## Development Notes

### 12/17/24

#### Option 1

break apart and treat each group uniquely (could be a good starting point)
- get it to WORK! can always go back in the future to streamline, backlog

#### Option 2
make library, use custom resampler for static (use .first()) #TODO
- good bc could modify library in future
- include comment in code 

general suggestions


include comment for each bloc, print statements in functions where necessary
remember that this will go public eventually and publish manuscript PLUS automating
process AND producing gridded product
SO aim for scripts that area ready for this high visibility! 
clear doc strings, "black formatting" (way to organize script)
future convos on standardization


commend in the script for future Vanessa! 
err on overcommenting 


use notes to yourself in Slack!!!! to reflect on script 

## Setup

In [30]:
import boto3
import geopandas as gpd
import numpy as np
import pandas as pd
import requests
import urllib
import datetime
import math
import shapely
import xarray as xr
import matplotlib.pyplot as plt
from io import BytesIO, StringIO
import scipy.stats as stats

import s3fs
import tempfile # Used for downloading (and then deleting) netcdfs to local drive from s3 bucket
import os
from shapely.geometry import Point
import time # Used for progress bar 
import sys # Used for progress bar 

# Silence warnings
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) # Warning is raised when creating Point object from coords. Can't figure out why. 

In [31]:
## Set AWS credentials
s3 = boto3.resource("s3")
s3_cl = boto3.client('s3') # for lower-level processes

## Set relative paths to other folders and objects in repository.
bucket_name = "wecc-historical-wx"
wecc_terr = "s3://wecc-historical-wx/0_maps/WECC_Informational_MarineCoastal_Boundary_land.shp"
wecc_mar = "s3://wecc-historical-wx/0_maps/WECC_Informational_MarineCoastal_Boundary_marine.shp"

In [32]:
# Define temporary directory in local drive for downloading data from S3 bucket
# If the directory doesn't exist, it will be created
# If we used zarr, this wouldn't be neccessary 
temp_dir = "./tmp"
if not os.path.exists(temp_dir): 
    os.mkdir(temp_dir)

## Define helper functions

In [33]:
def progressbar(it, prefix="", size=60, out=sys.stdout): # Python3.6+
    """
    Print a progress bar to console 

    References
    ----------
    https://stackoverflow.com/questions/3160699/python-progress-bar
    
    """
    count = len(it)
    start = time.time() # time estimate start
    def show(j):
        x = int(size*j/count)
        # time estimate calculation and string
        remaining = ((time.time() - start) / j) * (count - j)        
        mins, sec = divmod(remaining, 60) # limited to minutes
        time_str = f"{int(mins):02}:{sec:03.1f}"
        print(f"{prefix}[{u'█'*x}{('.'*(size-x))}] {j}/{count} Est wait {time_str}", end='\r', file=out, flush=True)
    show(0.1) # avoid div/0 
    for i, item in enumerate(it):
        yield item
        show(i+1)
    print("\n", flush=True, file=out)

In [34]:
def read_nc_from_s3(network_name, station_id, temp_dir):
    """Read netcdf file containing station data for a single station of interest from AWS s3 bucket 

    Parameters
    ----------
    network_name: str 
        Name of network (i.e. "ASOSAWOS")
        Must correspond with a valid directory in the s3 bucket (i.e. "CAHYDRO", "CDEC", "ASOSAWOS")
    station_id: str
        Station identifier; i.e. the name of the netcdf file in the bucket (i.e. "ASOSAWOS_72012200114.nc")
    
    Returns 
    -------
    station_data: xr.Dataset 
    
    Notes
    -----
    The data is first downloaded from AWS into a tempfile, which is then deleted after xarray reads in the file 
    I'd like to see us use a zarr workflow if possible to avoid this. 

    """

    # Temp file for downloading from s3
    temp_file = tempfile.NamedTemporaryFile(
        dir = temp_dir, 
        prefix = "", 
        suffix = ".nc",
        delete = True
    )

    # Create s3 file system 
    s3 = s3fs.S3FileSystem(anon=False)

    # Get URL to netcdf in S3
    s3_url = 's3://wecc-historical-wx/3_qaqc_wx_dev/{}/{}.nc'.format(network_name, station_id)

    # Read in the data using xarray 
    s3_file_obj = s3.get(s3_url, temp_file.name)
    station_data = xr.open_dataset(temp_file.name, engine='h5netcdf').load()

    # Close temporary file 
    temp_file.close()

    return station_data 

In [35]:
def qaqc_ds_to_df(ds, verbose=False):
    ## Add qc_flag variable for all variables, including elevation; 
    ## defaulting to nan for fill value that will be replaced with qc flag

    for key,val in ds.variables.items():
        if val.dtype==object:
            if key=='station':
                if str in [type(v) for v in ds[key].values]:
                    ds[key] = ds[key].astype(str)
            else:
                if str in [type(v) for v in ds.isel(station=0)[key].values]:
                    ds[key] = ds[key].astype(str)
                
    exclude_qaqc = ["time", "station", "lat", "lon", 
                    "qaqc_process", "sfcWind_method", 
                    "pr_duration", "pr_depth", "PREC_flag",
                    "rsds_duration", "rsds_flag", 
                    "anemometer_height_m",
                    "thermometer_height_m"
                   ] # lat, lon have different qc check

    raw_qc_vars = [] # qc_variable for each data variable, will vary station to station
    era_qc_vars = [] # our ERA qc variable
    old_era_qc_vars = [] # our ERA qc variable

    for var in ds.data_vars:
        if 'q_code' in var: 
            raw_qc_vars.append(var) # raw qc variable, need to keep for comparison, then drop
        if '_qc' in var: 
            raw_qc_vars.append(var) # raw qc variables, need to keep for comparison, then drop
        if '_eraqc' in var:
            era_qc_vars.append(var) # raw qc variables, need to keep for comparison, then drop
            old_era_qc_vars.append(var)

    print(f"era_qc existing variables:\n{era_qc_vars}")
    n_qc = len(era_qc_vars)
    
    for var in ds.data_vars:
        if var not in exclude_qaqc and var not in raw_qc_vars and "_eraqc" not in var:
            qc_var = var + "_eraqc" # variable/column label

            # if qaqc var does not exist, adds new variable in shape of original variable with designated nan fill value
            if qc_var not in era_qc_vars:
                print(f"nans created for {qc_var}")
                ds = ds.assign({qc_var: xr.ones_like(ds[var])*np.nan})
                era_qc_vars.append(qc_var)
    
    print("{} created era_qc variables".format(len(era_qc_vars)-len(old_era_qc_vars)))
    if len(era_qc_vars)!=n_qc:    
        print("{}".format(np.setdiff1d(old_era_qc_vars, era_qc_vars)))
    
    # Save attributes to inheret them to the QAQC'ed file
    attrs = ds.attrs
    var_attrs = {var:ds[var].attrs for var in list(ds.data_vars.keys())}

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=RuntimeWarning)
        df = ds.to_dataframe()

    # instrumentation heights
    if 'anemometer_height_m' not in df.columns:
        try:
            df['anemometer_height_m'] = np.ones(ds['time'].shape)*ds.anemometer_height_m
        except:
            print("Filling anemometer_height_m with NaN.", flush=True)
            df['anemometer_height_m'] = np.ones(len(df))*np.nan
        finally:
            pass
    if 'thermometer_height_m' not in df.columns:
        try:
            df['thermometer_height_m'] = np.ones(ds['time'].shape)*ds.thermometer_height_m
        except:
            print("Filling thermometer_height_m with NaN.", flush=True)
            df['thermometer_height_m'] = np.ones(len(df))*np.nan
        finally:
            pass

    # De-duplicate time axis
    df = df[~df.index.duplicated()].sort_index()
           
    # Save station/time multiindex
    MultiIndex = df.index
    station = df.index.get_level_values(0)
    df['station'] = station
    
    # Station pd.Series to str
    station = station.unique().values[0]
    
    # Convert time/station index to columns and reset index
    df = df.droplevel(0).reset_index()

    # Add time variables needed by multiple functions
    df['hour'] = pd.to_datetime(df['time']).dt.hour
    df['day'] = pd.to_datetime(df['time']).dt.day 
    df['month'] = pd.to_datetime(df['time']).dt.month 
    df['year'] = pd.to_datetime(df['time']).dt.year 
    df['date']  = pd.to_datetime(df['time']).dt.date
    
    return df#, MultiIndex, attrs, var_attrs, era_qc_vars 

## Precipitation Standardization

### load and explore sample precipication data

In [36]:
# load in single dc file from AWS
#ds = read_nc_from_s3('ASOSAWOS', 'ASOSAWOS_72494023234', temp_dir) #cleaned, but no sub-hourly precip data
ds = read_nc_from_s3('CRN', 'CRN_AGPC2', temp_dir) #contains sub-hourly precipitation data
#convert to formatted pandas dataframe
df = qaqc_ds_to_df(ds, verbose=False)

# check if precipittion data present
all_pr_vars = [var for var in df.columns if 'pr' in var]
all_pr_vars

era_qc existing variables:
['tas_eraqc', 'pr_5min_eraqc', 'pr_1h_eraqc', 'elevation_eraqc']
0 created era_qc variables


['pr_5min', 'pr_5min_qc', 'pr_1h', 'pr_5min_eraqc', 'pr_1h_eraqc']

## Break dataframe into four sub-dataframes

1. static variables: this include variables like elevation and station name, which stay the same through time
- take the first value to "replace" in the new time stamp
2. time variables: 
- Only the "time" column will be modified, to be the top of the hour. All others (day, month, year, date) are constant within the hour.
3. QC flags
- start off with treating them like strings, and concatenanting, with a comma
- There's an exception here: if the variable is flagged by _eraqc it should be ignored prior to standardization. So for example if precip at :01 is flagged, but precip at :06 is not, the obs at :06 is the one kept etc. We potentially might need to think on if there are qaqc flags that "are okay to keep" and one's that are not.
4. meteorological variables
- These will be resampled. 


In [110]:
# groupings of columns that fall into each category
# TODO: enentually add all possible variables - the below is just a subset based on the CRN network

constant_vars = ["time","station", "lat", "lon", "elevation",
                    "anemometer_height_m","thermometer_height_m"]

time_vars = ["time","hour","day","month","year","date"]

qaqc_vars = ["time","tas_qc", "tas_eraqc",
                    "pr_5min_eraqc","pr_1h_eraqc","pr_5min_qc","elevation_eraqc"]

met_vars = ["time","tas", "pr_5min","pr_1h"]


In [111]:
constant_df = df[constant_vars]
#constant_df.columns

time_df = df[time_vars] 
#time_df.columns

qaqc_df = df[qaqc_vars]
qaqc_df = qaqc_df[qaqc_vars].astype(str)
#qaqc_df.columns

met_df = df[met_vars] 
#met_df.columns

## Resampling

In [112]:
# resample meteorological variables 
met_result =  met_df.resample('1h',on='time').sum()
met_result


,tas,pr_5min,pr_1h
time,,,
2013-03-16 22:00:00,2255.260,0.0,0.0
2013-03-16 23:00:00,3377.180,0.0,0.0
2013-03-17 00:00:00,3360.760,0.0,0.0
2013-03-17 01:00:00,3332.630,0.0,0.0
2013-03-17 02:00:00,3313.460,0.0,0.0
...,...,...,...
2022-08-31 19:00:00,3656.360,0.0,0.0
2022-08-31 20:00:00,3675.440,0.0,0.0
2022-08-31 21:00:00,3671.684,0.0,0.0


In [113]:
# keep the first value in each hour
constant_result =  constant_df.resample('1h',on='time').first()
constant_result


,station,lat,lon,elevation,anemometer_height_m,thermometer_height_m
time,,,,,,
2013-03-16 22:00:00,CRN_AGPC2,40.155,-103.14167,1383.792,NaN,NaN
2013-03-16 23:00:00,CRN_AGPC2,40.155,-103.14167,1383.792,NaN,NaN
2013-03-17 00:00:00,CRN_AGPC2,40.155,-103.14167,1383.792,NaN,NaN
2013-03-17 01:00:00,CRN_AGPC2,40.155,-103.14167,1383.792,NaN,NaN
2013-03-17 02:00:00,CRN_AGPC2,40.155,-103.14167,1383.792,NaN,NaN
...,...,...,...,...,...,...
2022-08-31 19:00:00,CRN_AGPC2,40.155,-103.14167,1383.792,NaN,NaN
2022-08-31 20:00:00,CRN_AGPC2,40.155,-103.14167,1383.792,NaN,NaN
2022-08-31 21:00:00,CRN_AGPC2,40.155,-103.14167,1383.792,NaN,NaN


In [ ]:
qaqc_result = qaqc_df.resample('1h',on='time').sum() # .agg({qaqc_df: lambda x: ''.join(x)})
qaqc_result

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [65]:
concat_result["tas_qc"]

time
2013-03-16 22:00:00                nannannannannannannannan
2013-03-16 23:00:00    nannannannannannannannannannannannan
2013-03-17 00:00:00    nannannannannannannannannannannannan
2013-03-17 01:00:00    nannannannannannannannannannannannan
2013-03-17 02:00:00    nannannannannannannannannannannannan
                                       ...                 
2022-08-31 19:00:00    nannannannannannannannannannannannan
2022-08-31 20:00:00    nannannannannannannannannannannannan
2022-08-31 21:00:00    nannannannannannannannannannannannan
2022-08-31 22:00:00    nannannannannannannannannannannannan
2022-08-31 23:00:00    nannannannannannannannannannannannan
Freq: h, Name: tas_qc, Length: 82922, dtype: object

In [60]:
# keep the first value in each hour
time_result =  time_df.resample('1h',on='time').first()
time_result

,hour,day,month,year,date
time,,,,,
2013-03-16 22:00:00,22.0,16.0,3.0,2013.0,2013-03-16
2013-03-16 23:00:00,23.0,16.0,3.0,2013.0,2013-03-16
2013-03-17 00:00:00,0.0,17.0,3.0,2013.0,2013-03-17
2013-03-17 01:00:00,1.0,17.0,3.0,2013.0,2013-03-17
2013-03-17 02:00:00,2.0,17.0,3.0,2013.0,2013-03-17
...,...,...,...,...,...
2022-08-31 19:00:00,19.0,31.0,8.0,2022.0,2022-08-31
2022-08-31 20:00:00,20.0,31.0,8.0,2022.0,2022-08-31
2022-08-31 21:00:00,21.0,31.0,8.0,2022.0,2022-08-31


## Putting it all together

In [13]:
def precip_hourly_standardization(df):
    
    """
    
    Rules:
    ------
        1) pr_5min < pr_1h < pr_24h
        2) pr_localmid should never exceed pr_24h

    Input:
    ------
        df [pd.DataFrame]: station dataset converted to dataframe through QAQC pipeline

    Output:
    -------
        if success:
            df [pd.DataFrame]: QAQC dataframe with additional column "pr_1hr"
        if failure:
            None
    """

    print("Running: precip_hourly_standardization", flush=True)
    
    # identify which precipitation vars are reported 
    all_pr_vars = [var for var in df.columns if 'pr' in var] # can be variable length depending if there is a raw qc var
    pr_vars = [var for var in all_pr_vars if not any(True for item in vars_to_remove if item in var)] # remove all qc variables so they do not also run through: raw, eraqc, qaqc_process
    
    # break dataframe into four sub-dataframes
    # category 1: 

    # TODO: add to log file
    try:        
        # if station does not report any precipitation values, or only one, bypass
        if len(pr_vars) == 0:
            print('Station does not report precipitation variables - bypassing hourly aggregation', flush=True)
            return df
        if 'pr_1h' in pr_vars and 'pr_5min' not in pr_vars:
            print('Station already reports hourly precipitation - bypassing hourly aggregation', flush=True)
            return df
        if 'pr_24h' in pr_vars and 'pr_5min' not in pr_vars:
            print('Station reports daily precipitation - bypassing hourly aggregation', flush=True)
            return df
        if 'pr_5min' in pr_vars and 'pr_24h' not in pr_vars and 'pr_1h' not in pr_vars:
            print('attempting hourly aggregation', flush=True)
            df = df.resample('1h',on='time').sum()
            return df
    
    except Exception as e:
        print("precip_hourly_standardization failed with Exception: {0}".format(e), flush=true)
        return None

In [ ]:
result = precip_hourly_standardization(df)
result

Running: precip_hourly_standardization


## Code Sandbox

In [ ]:

# agg_funcs = {
#     col:'sum' if col in met_vars else continue
#     for col in df.columns
# }

#df_resampled = df.resample('D').agg(agg_funcs)

# result = df.resample('1h',on='time').add_funcs()
# result

In [ ]:
# Sample DataFrame
df = pd.DataFrame({
    'date': pd.date_range('2023-01-01', periods=10, freq='D'),
    'category': ['A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'B'],
    'value': [10, 15, 12, 18, 14, 20, 16, 22, 18, 24]
})
df.set_index('date', inplace=True)

# Group by the variable and apply different resampling techniques
resampled_df = df.groupby('category').resample('W').agg({
    'value': {'A': 'mean', 'B': 'first'}
})